# OpenRouteService testing by Python Bindings

https://github.com/GIScience/openrouteservice-py

In [24]:
import openrouteservice as ors
import folium
import json
from pprint import pprint

povo_center = [46.0682, 11.1551]
povo_zoom = 16
povo_start = []

mezzo_center = [46.2147, 11.1065]
mezzo_zoom = 14
mezzo_start = [46.2170,11.1066]

#contains two features MultiPoint
povo_points = json.load(open('./data/povo.geojson'))['features']
mezzo_points = json.load(open('./data/mezzocorona.geojson'))['features']


# OpenRouteService Optimization Routing examples

This notebook is just a small reference on how to use `openrouteservice` package to request:
- [directions](https://openrouteservice.org/dev)
- [Route Optimization](https://openrouteservice.org/dev/#/api-docs/optimization/post)

In [25]:
client = ors.Client(base_url='http://ors:8080/ors', key='')

center = mezzo_center
zoom = mezzo_zoom
points = mezzo_points

In [26]:
points = {
    'A': points[0]['geometry']['coordinates'],
    'B': points[1]['geometry']['coordinates']
}

pprint(points)

{'A': [[11.095612049102785, 46.21736181569706],
       [11.092135906219482, 46.216560067112056],
       [11.09722137451172, 46.21403596735467]],
 'B': [[11.114044189453127, 46.21506046889148],
       [11.118743419647217, 46.21494168708413],
       [11.116554737091064, 46.21342719651941],
       [11.115074157714844, 46.211318718540376]]}


## Directions

In [27]:
M = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)

routeA = client.directions(
    coordinates = points['A'],
    profile = 'driving-car',
    format = 'geojson',
    validate = False
)

routeB = client.directions(
    coordinates = points['B'],
    profile = 'driving-car',
    format = 'geojson',
    validate = False
)

polyA = [list(reversed(coord)) for coord in routeA['features'][0]['geometry']['coordinates']]
polyB = [list(reversed(coord)) for coord in routeB['features'][0]['geometry']['coordinates']]

folium.PolyLine(polyA, color='red').add_to(M)
folium.PolyLine(polyB, color='blue').add_to(M)
#pprint(points['A'])

for c in points['A']:
    m = list(reversed(c))
    folium.Marker(m, icon=folium.Icon(color='red')).add_to(M)

for c in points['B']:
    m = list(reversed(c))
    folium.Marker(m).add_to(M)
M

## VRP Vehicle Routing Problem 

Optimize job scheduling for multiple vehicles.
The next example makes more use of the power of the ORS optimization endpoint. Two vehicles are assigned to 6 jobs, where the capacity constraints are such that each vehicle can only carry out 3 jobs on its route.

In [24]:
m = folium.Map(location=povo_center, tiles='OpenStreetMap', zoom_start=zoom)

#punto di carico uguale al punto di scarico/parcheggio
vehicle_locations = [mezzo_start, mezzo_start]

job_locations = [points[1]]

# Assign vehicles to do the jobs
vehicles = []
for idx, coords in enumerate(vehicle_locations):
    vehicles.append(ors.optimization.Vehicle(
        id =idx,
        profile='driving-hgv',
        start=coords,
        end=coords,
        capacity=[3]  # Limit capacity so only 3 jobs can be taken by each vehicle
    ))
    folium.Marker(location=list(reversed(coords)), icon=folium.Icon(icon='truck', prefix='fa')).add_to(m)

# Define jobs to be carried out
jobs=[]
for idx, coords in enumerate(job_locations):
    jobs.append(ors.optimization.Job(
        id=idx,
        location=coords,
        amount=[1]  # Occupies capacity in vehicle
    ))
    folium.Marker(location=list(reversed(coords)), icon=folium.Icon(icon='archive', prefix='fa', color='green')).add_to(m)

optimized = client.optimization(
    jobs=jobs,
    vehicles=vehicles,
    geometry=True,  ## will output the geometry,
)

folium.PolyLine(
    locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(optimized['routes'][0]['geometry'])['coordinates']],
    color='red'
).add_to(m)

folium.PolyLine(
    locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(optimized['routes'][1]['geometry'])['coordinates']],
    color='orange'
).add_to(m)

m